In [1]:
import gensim
from gensim.models import KeyedVectors
import random
import re

# compare two models for a word

In [2]:
def agreement(word,m1,m2,num):
    sim1=m1.most_similar(positive=[word], topn=num)
    sim2=m2.most_similar(positive=[word], topn=num)
    agreements=[]
    for (w,c) in sim1:
        found=0
        for(w2,c2) in sim2:
            if w == w2:
                found=1
        if found == 1:
            agreements.append(w)
    return(agreements)

# load models

In [3]:
m1=gensim.models.word2vec.Word2Vec.load('dewiki190420.word2vec.sg.hs.324.10')
wv_sg=m1.wv
del m1

In [4]:
m2=gensim.models.word2vec.Word2Vec.load('dewiki190420.word2vec.324.10')
wv_cbow=m2.wv
del m2

In [6]:
m3=KeyedVectors.load_word2vec_format("word2vec.txt")
wv_glove=m3.wv
del m3

# the first and very long running comparison

In [ ]:
agreements=[]
i=0
while i<11:
    agreements.append(0)
    i+=1

with open('agreements.log','wt',encoding='utf-8') as f:
    i=0
    for token in m1.wv.vocab:
        a=agreement(token,m1.wv,m2.wv,10)
        if i%1000==0:
            print("{}\t{}".format(i,agreements))
        f.write("{}\t{}\n".format(token,a))
        agreements[len(a)]+=1
        i+=1

# convert frequency word list into frequency class wordlist and prepare the lookup structures

In [ ]:
wc_pat=re.compile(" *(\d+)\t(.*)");
wf_hkl={}
hklwl=[]
i=0;
while i<22:
    hklwl.append([])
    i+=1
with open("types.hkl",encoding='utf-8') as f:
    for line in f:
        l=wc_pat.match(line)
        wf_hkl.update({l.group(2): int(l.group(1))})
        hklwl[int(l.group(1))].append(l.group(2))

# calculate comparisons for one frequency class

In [17]:
import random
def test(wv1,wv2,wv3,hkl,num,topn):
    counts1=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    counts2=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    counts3=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    counts4=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    counts5=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    counts6=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for i in range(num-1):
        good=0
        while good < 1:
            testword=hklwl[hkl][random.randint(0,len(hklwl[hkl]) - 1)]
            if testword in wv1.vocab and testword in wv2.vocab and testword in wv3.vocab:
                good=1
        #print('{}\t{}'.format(testword, wf_hkl[testword]),end='')
        for (word,sim) in wv1.most_similar(positive=[testword],topn=topn):
            #print('\t{}'.format(wf_hkl[word]),end='')
            try:
                counts1[wf_hkl[word]]+=1
            except:
                #key error
                print('',end='')
        #print("")
        for (word,sim) in wv2.most_similar(positive=[testword],topn=topn):
            #print('\t{}'.format(wf_hkl[word]),end='')
            try:
                counts2[wf_hkl[word]]+=1
            except:
                #key error
                print('',end='')
        for (word,sim) in wv3.most_similar(positive=[testword],topn=topn):
            #print('\t{}'.format(wf_hkl[word]),end='')
            try:
                counts3[wf_hkl[word]]+=1
            except:
                #key error
                print('',end='')
        for word in agreement(testword,wv1,wv2,topn):
            try:
                counts4[wf_hkl[word]]+=1
            except:
                #key error
                print('',end='')
        #print("")
        for word in agreement(testword,wv1,wv3,topn):
            try:
                counts5[wf_hkl[word]]+=1
            except:
                #key error
                print('',end='')
    
        for word in agreement(testword,wv2,wv3,topn):
            try:
                counts6[wf_hkl[word]]+=1
            except:
                #key error
                print('',end='')
    return counts1,counts2,counts3,counts4,counts5,counts6

# collect results for all three models and all comparisons

In [ ]:
counts_glove=[]
counts_sg=[]
counts_cbow=[]
counts_agre_g_s=[]
counts_agre_g_c=[]
counts_agre_s_c=[]
for hkl in range(6,22):
    print(hkl)
    (counts1,counts2,counts3,counts4,counts5,counts6)=test(wv_glove,wv_sg,wv_cbow,hkl,1000,10)
    counts_glove.append([hkl] + counts1[6:])
    counts_sg.append([hkl] + counts2[6:])
    counts_cbow.append([hkl] + counts3[6:])
    counts_agre_g_s.append([hkl] + counts4[6:])
    counts_agre_g_c.append([hkl] + counts5[6:])
    counts_agre_s_c.append([hkl] + counts6[6:])

# output a sample result

In [ ]:
for c in counts_agre_s_c:
    for v in c:
        print("\t{}".format(v),end='')
    print("")

# Get data for frequency class plot

In [210]:
for i,hk in enumerate(hklwl):
    print("{}\t{}".format(i,len(hk)))

0	0
1	0
2	0
3	0
4	0
5	0
6	175
7	212
8	404
9	828
10	1525
11	2760
12	4896
13	8441
14	14524
15	24782
16	43470
17	73503
18	124631
19	211042
20	330124
21	500728
